## SNR calculation for Plomp sentences based on SII for SPIN, for all subjects (N=20)

13 may 2022 Guido Cattani

In [1]:
import numpy as np
import pandas as pd

In [2]:
import glob
from pathlib import Path

In [3]:
def list_files():
    dr = '/media/guido/LACIE/Cingle_Guido/Master/Implant/Calculated_data/SNR/'
    l = glob.glob(dr + 'SII_*.csv')
    return l

In [4]:
def extract_snr(f_in):   
    digits = list(''.join(filter(str.isdigit, f_in)))
    speech_level = int(digits[0] + digits[1])
    noise_level = int(digits[2] + digits[3])
    snr = speech_level - noise_level
    return snr

In [5]:
def read_sii_spin(f_in):
    # function to read single file sii spin
    p_in = Path(f_in)   
    df = pd.read_csv(p_in)
    df.set_index('Study_ID', inplace=True)
    s = df['AC&BC_path']
    return s

In [6]:
def sii_snr():    
    l = list_files()
    d = dict()
    for fin in l:
        data = read_sii_spin(fin)
        snr = extract_snr(fin)
        d.update({snr : data})
    df = pd.DataFrame(d)
    cl = list(df.columns)
    cl.sort()
    df = df[cl]
    return df

In [7]:
def to_interpolate():
    data = sii_snr() #sii dataframe of subjects

    sii50 = 0.3373   # sii value corresponding with a score of 0.5 voor SPIN / Plomp sentences

    # make a dataframe with target values
    target = data.copy()
    target = (target ** 0) * sii50

    # make filters and filter the sii dataframe
    filter_big = (data > target)
    filtered_big = filter_big * data
    filtered_big.replace(to_replace=0, value=1, inplace=True)
    supValues = filtered_big.min(axis = 1)
    supValueIndex = filtered_big.idxmin(axis = 1)

    filter_small = (data <= target)
    filtered_small = filter_small * data
    infValues = filtered_small.max(axis = 1)
    infValueIndex = filtered_small.idxmax(axis = 1)

    # make the dataframe with the data to calculate the S/N by interpolation
    together = [infValueIndex, supValueIndex, infValues, supValues]
    result = pd.concat(together, axis=1)
    d = {0:'infLevel', 1:'supLevel', 2:'infSII', 3:'supSII'}
    result.rename(d, axis=1, inplace=True)
    return result

In [8]:
def snr_interpolation():
    df = to_interpolate()
    sii50 = 0.3373   # sii value corresponding with a score of 0.5 voor SPIN / Plomp sentences
    slope = (df['supLevel'] - df['infLevel']) / (df['supSII'] - df['infSII'])
    snr = df['infLevel'] + (sii50 - df['infSII']) * slope
    snr = round(snr, 1)
    return snr

In [9]:
def presentation_results():
    data = to_interpolate()
    snr  = snr_interpolation()
    data.insert(1, column='SNR_predicted', value=snr)
    return data

In [10]:
def write_results_to_file():
    snr  = snr_interpolation()
    df = pd.DataFrame(snr)
    df.rename({0:'SNR_predicted'}, axis=1, inplace=True)
    p = Path('/media/guido/LACIE/Cingle_Guido/Master/Implant/Calculated_data/SNR/SNR_prediction.csv')
    df.to_csv(p)

In [11]:
write_results_to_file()

In [12]:
presentation_results()

,infLevel,SNR_predicted,supLevel,infSII,supSII
Study_ID,,,,,
34,-4,-3.3,-3,0.3230,0.3432
35,-3,-3.0,-2,0.3362,0.3590
43,2,2.6,3,0.3208,0.3482
47,-6,-5.0,-5,0.3166,0.3378
50,3,3.3,4,0.3294,0.3571
53,3,3.2,4,0.3331,0.3579
54,-7,-6.1,-6,0.3177,0.3388
56,-7,-6.7,-6,0.3300,0.3532
59,3,3.6,4,0.3206,0.3480
